In [1]:
from preprocessing import *
from sklearn.model_selection import train_test_split, KFold, ParameterGrid
import argparse
from model import *
from train import test
import torch.optim as optim

from MatrixVectorizer import *


In [2]:
# load csvs as numpy
lr_data_path = '../data/lr_train.csv'
hr_data_path = '../data/hr_train.csv'

lr_train_data = np.loadtxt(lr_data_path, delimiter=',')
hr_train_data = np.loadtxt(hr_data_path, delimiter=',')
lr_train_data[lr_train_data < 0] = 0
np.nan_to_num(lr_train_data, copy=False)

hr_train_data[hr_train_data < 0] = 0
np.nan_to_num(hr_train_data, copy=False)

# map the anti-vectorize function to each row of the lr_train_data

lr_train_data_vectorized = np.array([MatrixVectorizer.anti_vectorize(row, 160) for row in lr_train_data])
hr_train_data_vectorized = np.array([MatrixVectorizer.anti_vectorize(row, 260) for row in hr_train_data])


In [3]:

# lr_train into test and training set (training set passed to kfold for cross validation)
test_proportion = 0.2
lr_train, lr_test, hr_train, hr_test = train_test_split(lr_train_data_vectorized, hr_train_data_vectorized, test_size=test_proportion, random_state=42)

In [4]:
print(f"{lr_train_data_vectorized.shape=}")
print(f"{lr_train.shape=}")
print(f"{lr_test.shape=}")

print(f"{hr_train_data_vectorized.shape=}")
print(f"{hr_train.shape=}")
print(f"{hr_test.shape=}")



lr_train_data_vectorized.shape=(168, 160, 160)
lr_train.shape=(134, 160, 160)
lr_test.shape=(34, 160, 160)
hr_train_data_vectorized.shape=(168, 260, 260)
hr_train.shape=(134, 260, 260)
hr_test.shape=(34, 260, 260)


In [5]:
cv = KFold(n_splits=3, random_state=42, shuffle=True)

In [6]:
criterion = nn.L1Loss()

def train(model, optimizer, lr_train, hr_train, args):
  
  all_epochs_loss = []
  no_epochs = args.epochs
  model.train()

  for epoch in range(no_epochs):
    epoch_loss = []
    epoch_error = []

    for lr,hr in zip(lr_train, hr_train):      
      lr = torch.from_numpy(lr).type(torch.FloatTensor)
      hr = torch.from_numpy(hr).type(torch.FloatTensor)
      
      model_outputs,net_outs,start_gcn_outs,layer_outs = model(lr)
      # model_outputs  = unpad(model_outputs, args.padding)
      # weights = unpad(model.layer.weights, args.padding)
      
      padded_hr = pad_HR_adj(hr,args.padding)
      eig_val_hr, U_hr = torch.linalg.eigh(padded_hr, UPLO='U')
      
      # loss = criterion(net_outs, start_gcn_outs) + criterion(model.layer.weights,U_hr) + args.lmbda * criterion(model_outputs, hr) 
      loss = args.lmbda * criterion(net_outs, start_gcn_outs) + criterion(model.layer.weights,U_hr) + criterion(model_outputs, padded_hr) 

      
      error = criterion(model_outputs, padded_hr)
      
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      epoch_loss.append(loss.item())
      epoch_error.append(error.item())
      
    print("Epoch: ",epoch+1, "Loss: ", np.mean(epoch_loss), "Error: ", np.mean(epoch_error))
    all_epochs_loss.append(np.mean(epoch_loss))

In [7]:
# Define sets of hyperparameters for tuning
learning_rates = [0.0001, 0.0002]
lambdas = [1000, 5000, 10000]
hidden_dims = [256, 512, 1024]

num_splt = 3
epochs = 10
# lr = 0.00005
# lmbda = 5000
lr_dim = 160
hr_dim = 320
# hidden_dim = 512
padding = 30
ks = [0.7, 0.3]

hyperparam_grid = {
    'lr': learning_rates,
    'lmbda': lambdas,
    'hidden_dim': hidden_dims
}

best_hyperparams = None
best_val_loss = float('inf')

for hyperparams in ParameterGrid(hyperparam_grid):
    print(f"Training with hyperparams: {hyperparams}")

    args = argparse.Namespace(epochs=epochs,lmbda = hyperparams['lmbda'], lr_dim=lr_dim, hr_dim=hr_dim, hidden_dim = hyperparams['hidden_dim'], padding=padding)
    
    model = GSRNet(ks, args)
    optimizer = optim.Adam(model.parameters(), lr=hyperparams['lr'])
    
    cv_loss = []
    for train_index, val_index in cv.split(lr_train):
        lr_train_fold, lr_val_fold = lr_train[train_index], lr_train[val_index]
        hr_train_fold, hr_val_fold = hr_train[train_index], hr_train[val_index]

        train(model, optimizer, lr_train_fold, hr_train_fold, args)
        val_loss = test(model, lr_val_fold, hr_val_fold, args)
        cv_loss.append(val_loss)
    print(cv_loss)
    avg_val_loss = np.mean(cv_loss)
    print(f"Validation loss: {avg_val_loss}")
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_hyperparams = hyperparams

print(f"Best hyperparameters: {best_hyperparams}")

Training with hyperparams: {'hidden_dim': 256, 'lmbda': 1000, 'lr': 0.0001}


Epoch:  1 Loss:  135.62602110123368 Error:  124.54854009811127
Epoch:  2 Loss:  129.06227593743398 Error:  124.53123877272847
Epoch:  3 Loss:  128.63830950554836 Error:  124.52418454250927
Epoch:  4 Loss:  128.25831981991115 Error:  124.52103321664454
Epoch:  5 Loss:  127.91880815484551 Error:  124.51928923965505
Epoch:  6 Loss:  127.61912836921348 Error:  124.51848003301728
Epoch:  7 Loss:  127.3566579657994 Error:  124.51769013627526
Epoch:  8 Loss:  127.13027440831902 Error:  124.51748274342063
Epoch:  9 Loss:  126.93418893653356 Error:  124.51687634133556
Epoch:  10 Loss:  126.76505933718735 Error:  124.51635145270423
Test error MAE:  0.18867355518870885
Epoch:  1 Loss:  126.63289821951577 Error:  124.51638979858227
Epoch:  2 Loss:  126.49974152211392 Error:  124.51635146559624
Epoch:  3 Loss:  126.38148059603873 Error:  124.51571641150821
Epoch:  4 Loss:  126.27705239312033 Error:  124.51558829107312
Epoch:  5 Loss:  126.18570566445254 Error:  124.51601550292767
Epoch:  6 Loss:  1

In [9]:
# run 'hidden_dim': 256, 'lmbda': 1000, 'lr': 0.0001 on test set
args = argparse.Namespace(epochs=epochs,lmbda = best_hyperparams['lmbda'], lr_dim=lr_dim, hr_dim=hr_dim, hidden_dim = best_hyperparams['hidden_dim'], padding=padding)
model = GSRNet(ks, args)
# train on full training set
optimizer = optim.Adam(model.parameters(), lr=best_hyperparams['lr'])
train(model, optimizer, lr_train, hr_train, args)
# test on test set
test_loss = test(model, lr_test, hr_test, args)


Epoch:  1 Loss:  91.82829734460631 Error:  82.77363934323414
Epoch:  2 Loss:  86.90207556112489 Error:  82.75784606230793
Epoch:  3 Loss:  86.4174696972121 Error:  82.75415973726716
Epoch:  4 Loss:  85.97195963005521 Error:  82.75309094703242
Epoch:  5 Loss:  85.56911781296802 Error:  82.75169043415296
Epoch:  6 Loss:  85.22096520662308 Error:  82.75080910575257
Epoch:  7 Loss:  84.93222978221836 Error:  82.75081919711917
Epoch:  8 Loss:  84.69876960587146 Error:  82.75068719963085
Epoch:  9 Loss:  84.51116934374197 Error:  82.75057061289006
Epoch:  10 Loss:  84.36385146094791 Error:  82.75011517190889
Test error MAE:  0.18168841214740977
